In [12]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import os
import shutil
import sys
import logging
import glob
import numpy as np
from itertools import product
import gc

#import pyarrow

In [13]:
client = Client("tcp://127.0.0.1:9100")

In [14]:
client

<Client: 'tcp://127.0.0.1:9100' processes=4 threads=4, memory=18.63 GiB>

In [15]:
os.getcwd()

'/mnt/labshares/schumm-lab/bryanb/teststate/scripts'

In [16]:
# NOTE: July 2022: Using CA here because they have some de files other states omit
state_dir="../../data2/medicaid/2016/CA/taf"

In [17]:
# name of fake state (set in create_state.sh)
new_state_name="TS2"

# section files (CHANGE THESE TO GENERATE DIFFERENT SECTIONS)
section='de'
filenames=['debse','dedsb','dedts','dehsp','demc','demfp','dewvr']

section_dir=f'{state_dir}/{section}'

# Create empty dataframes with file names
file_vars=[]
for file in filenames:
    vars()[file] = pd.DataFrame()
    file_vars.append(vars()[file])
    
# Add medicaid data to the dataframes
for i in range(len(file_vars)):
    file_vars[i]=pd.read_parquet(f'{section_dir}/{filenames[i]}/parquet/part.0.parquet', engine = 'pyarrow')

In [18]:
# Drop data and create empty dataframes containing just the columns
for i in range(len(file_vars)):
    vars()[filenames[i] + '_empty']=file_vars[i].drop(file_vars[i][file_vars[i].index >= '0'].index)
    print(f'Testing {filenames[i] + "_empty"} is empty:')
    assert(vars()[filenames[i] + '_empty'].empty)
    
# save files as empty fake state data
for i in range(len(file_vars)):
    print(f'Saving {filenames[i] + "_empty"} as {filenames[i] + ".csv"}')
    vars()[filenames[i] + "_empty"].to_csv(f'{filenames[i]}.csv')


Testing debse_empty is empty:
Testing dedsb_empty is empty:
Testing dedts_empty is empty:
Testing dehsp_empty is empty:
Testing demc_empty is empty:
Testing demfp_empty is empty:
Testing dewvr_empty is empty:
Saving debse_empty as debse.csv
Saving dedsb_empty as dedsb.csv
Saving dedts_empty as dedts.csv
Saving dehsp_empty as dehsp.csv
Saving demc_empty as demc.csv
Saving demfp_empty as demfp.csv
Saving dewvr_empty as dewvr.csv


In [19]:
# move files to their appropriate location
for filename in filenames:
    shutil.move(f'{filename}.csv', f'../{new_state_name}/taf/{section}/{filename}/csv/{filename}.csv')
    print(f'moving {filename} to ../{new_state_name}/taf/{section}/{filename}/csv/{filename}.csv')

moving debse to ../TS2/taf/de/debse/csv/debse.csv
moving dedsb to ../TS2/taf/de/dedsb/csv/dedsb.csv
moving dedts to ../TS2/taf/de/dedts/csv/dedts.csv
moving dehsp to ../TS2/taf/de/dehsp/csv/dehsp.csv
moving demc to ../TS2/taf/de/demc/csv/demc.csv
moving demfp to ../TS2/taf/de/demfp/csv/demfp.csv
moving dewvr to ../TS2/taf/de/dewvr/csv/dewvr.csv
